## sc3nb test file

In [17]:
import sc3nb as scn
import time

In [4]:
sc = scn.startup(sclangpath="/Applications/SuperCollider-3.9/SuperCollider.app/Contents/MacOS/sclang")
# add startup() first arg to be filename if sclang is elsewhere
# sc = sc3nb.startup("/Applications/Supercollider/SuperCollider.app/Contents/MacOS/sclang", discard_output=True, verbose=True)
# two tones should play after approx 3-5s, if not, try again...

Starting sclang...
Done.
Registering UDP callback...
Done.
Booting server...
Done.
sc3nb started


In [6]:
# scope window should open
%scv s.scope

In [7]:
# tone should play, output should be "-> Synth('default': ####)" 
# shortcut CMD-. should stop it, if not use line below
%scv Synth(\default)

In [8]:
%sc s.freeAll

In [10]:
# should give 206
scn.linlin(3, 0, 10, 200, 220)

206.0

In [11]:
help(scn.linlin)

Help on function linlin in module sc3nb.helpers:

linlin(x, smi, sma, dmi, dma)
    TODO
    
    Arguments:
        x {float} -- [description]
        smi {float} -- [description]
        sma {float} -- [description]
        dmi {float} -- [description]
        dma {float} -- [description]
    
    Returns:
        float -- [description]



In [13]:
%%scv
SynthDef("s1", { | out=0, freq=400, amp=0.5, gate=1|
    Out.ar(out, SinOsc.ar(freq, mul: amp)*EnvGen.kr(Env.adsr(0,0,1,4), gate, doneAction: 2)!2)
}).add()

In [14]:
%sc x = Synth.new(\s1)

In [15]:
%sc x.release  // sound should fade out

In [18]:
# sound should start 2s after code execution and stop 2s later
now = time.time()
sc.bundle(now+1, "/s_new", ["s1", 1100, 0, 0])
sc.bundle(now+2, "/n_set", [1100, "gate", 0])

In [19]:
# sc.exit()  # to close sc3, for some reason del(sc) does not work...

## sc3 recording functions

In [20]:
# either use absolute time
sc.prepare_for_record(0, "record.wav", 9, 2, "wav", "int16")
now = time.time()  # use 0 for relative time (less precise)
sc.record(now)
sc.bundle(now+0.1, "/s_new", ["s1", 1100, 0, 0])
sc.bundle(now+0.2, "/n_set", [1100, "freq", 600])
sc.bundle(now+0.3, "/n_set", [1100, "gate", 0])
sc.stop_recording(now+4)  # to pause, in case you want to resume with record()

## simpleaudio record and play functions

In [22]:
sig = scn.record(2.5, 1, 44100, 1024)

In [23]:
scn.play(sig, 1, sr=44100)

## guitar synth - simple

In [25]:
%%scv
SynthDef(\guitar, { | freq=110, amp=0.5, dur=1.5| 
    var sig, env;
    sig = DynKlang.ar(`[ [110, 220, 329], [-12.6, -28.6, -50.7].dbamp, nil], freq/110.0, 0);
    env = EnvGen.kr(Env.perc(0.001, dur, 1, -4), amp, doneAction: 2);
    Out.ar(0, (env*sig)!2);
}).add()

In [29]:
%%scv 
x = Synth.new(\guitar, [\freq, 110, \dur, 6]);
x.set(\freq, 220)

In [28]:
%%scv
{(52+[0,2,4,7,9,7,4,2,0]).midicps.do{ |f| 
    x = Synth.new(\guitar, [\freq, 2*f, \dur, 2.5]);
    0.25.wait;
}}.fork